In [ ]:
# Imports
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(143)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Get Dataset
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
testset = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

In [ ]:
# Inspect one image from the dataset
img, label = trainset.__getitem__(0)
img.size()

def imshow(img):
    # Expects an img in tensor -1 to 1 range
    #img = (img + 1)/2.0
    npimg = img.squeeze().numpy()
    plt.imshow(npimg)
    plt.show()

imshow(img)
print(label)

In [ ]:
# Create a data loader for dataset
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                           shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                           shuffle=False, num_workers=2)

In [ ]:
# Create the encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 5, 3)
        self.conv2 = nn.Conv2d(5, 13, 3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(13*5*5, 20)  # 5x5 from the image
        self.fc2 = nn.Linear(13*5*5, 20)  # Final latent dimension is 20
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 13*5*5)
        mu = self.fc1(x)
        logvar = self.fc2(x)
        return mu, logvar

# Creates the decoder class
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(20, 120)
        self.fc2 = nn.Linear(120, 784)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.sigmoid(x)

# Creates the VAE class
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        
    def reparam(self, mu, logvar):
        sigma = torch.exp(0.5*logvar)
        eps = torch.rand_like(sigma)
        return mu + eps*sigma
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparam(mu, logvar)
        x = self.decoder(z)
        return x, mu, logvar

In [ ]:
# Define the loss function involved
def loss_func(x, recon_x, mu, logvar):
    recon_loss = F.binary_cross_entropy(recon_x, x.view(-1, 784))
    KL_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + KL_loss

In [ ]:
# Define the training process
def train(epoch, model, optimizer, train_loader):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon, mu, logvar = model(data)
        loss = loss_func(data, recon, mu, logvar)
        loss.backward()
        train_loss += loss
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train_epoch: {}\t[{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset), 
                loss.item()/len(data)))
    print('Epoch: {} Avg train loss = {:.6f}'.format(
        epoch, train_loss/len(train_loader.dataset)))

In [ ]:
# Define the test process
def test(epoch, model, test_loader):
    model.eval()
    test_loss = 0
    for data, _ in test_loader:
        data = data.to(device)
        recon, mu, logvar = model(data)
        loss = loss_func(data, recon, mu, logvar)
        test_loss += loss
    test_loss /= len(test_loader.dataset)
    print('Epoch: {} Avg test loss: {:.6f}'.format(epoch, test_loss))
    return test_loss

In [ ]:
model = VAE().to(device)
model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, 10):
    train(epoch, model, optimizer, train_loader)
    test_loss = test(epoch, model, test_loader)